In [27]:
from neo4j.v1 import GraphDatabase, basic_auth
import pandas as pd

In [7]:
driver = GraphDatabase.driver("bolt://34.207.189.97:33619", auth=basic_auth("neo4j", "gulfs-possibility-contribution"))

In [21]:
create_links_graph_query = """\
CREATE (home:Page{name:'Home'})
,(about:Page{name:'About'})
,(product:Page{name:'Product'})
,(links:Page{name:'Links'})
,(a:Page{name:'Site A'})
,(b:Page{name:'Site B'})
,(c:Page{name:'Site C'})
,(d:Page{name:'Site D'})

CREATE (home)-[:LINKS]->(about)
,(about)-[:LINKS]->(home)
,(product)-[:LINKS]->(home)
,(home)-[:LINKS]->(product)
,(links)-[:LINKS]->(home)
,(home)-[:LINKS]->(links)
,(links)-[:LINKS]->(a)-[:LINKS]->(home)
,(links)-[:LINKS]->(b)-[:LINKS]->(home)
,(links)-[:LINKS]->(c)-[:LINKS]->(home)
,(links)-[:LINKS]->(d)-[:LINKS]->(home)
"""

with driver.session() as session:
    result = session.write_transaction(lambda tx: tx.run(create_links_graph_query))
    print(result.consume().metadata["stats"])

{'labels-added': 8, 'relationships-created': 14, 'nodes-created': 8, 'properties-set': 8}


In [35]:
streaming_query = """\
CALL algo.pageRank.stream('Page', 'LINKS', {iterations:20, dampingFactor:0.85})
YIELD node, score
RETURN node.name as page,score 
ORDER BY score DESC
LIMIT 20
"""

with driver.session() as session:
    result = session.read_transaction(lambda tx: tx.run(streaming_query))        
    df = pd.DataFrame([dict(zip(r.keys(), r.values())) for r in result])    

In [36]:
df

,page,score
0,Home,3.233877
1,About,1.060435
2,Product,1.060435
3,Links,1.060435
4,Site A,0.329035
5,Site B,0.329035
6,Site C,0.329035
7,Site D,0.329035


In [40]:
write_results_query = """\
CALL algo.pageRank('Page', 'LINKS', {iterations:20, dampingFactor:0.85, write: true,writeProperty:"pagerank"})
YIELD nodes, iterations, loadMillis, computeMillis, writeMillis, dampingFactor, write, writeProperty
"""

with driver.session() as session:
    result = session.read_transaction(lambda tx: tx.run(write_results_query))        
    df = pd.DataFrame([dict(zip(r.keys(), r.values())) for r in result])

df

,computeMillis,dampingFactor,iterations,loadMillis,nodes,write,writeMillis,writeProperty
0,0,0.85,20,1,8,True,0,pagerank
